In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 读取训练数据, 训练集包含500个模型结构，以及这些结构在cplfw，market1501，dukemtmc等8个任务上的性能排序
import json
with open('./data/CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)
print(train_data['arch1'])
print('train_num:',len(train_data.keys()))

{'cplfw_rank': 9, 'market1501_rank': 361, 'dukemtmc_rank': 426, 'msmt17_rank': 433, 'veri_rank': 327, 'vehicleid_rank': 480, 'veriwild_rank': 425, 'sop_rank': 367, 'arch': 'l231131331121121331111211121331321321'}
train_num: 500


In [3]:
import pandas as pd
import numpy as np

target_cols = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']

def get_df(train_data):
    ret = []
    for k, v in train_data.items():
        tmp = list(v['arch'])
        tmp1 = []
        for c in target_cols:
            tmp1.append(v[c])
        ret.append(tmp+tmp1+[k,v['arch']])
    retf = pd.DataFrame(ret,columns=[f'col{_}' for _ in range(len(tmp))]+target_cols+['id','arch'])
    retf['col0'] = retf['col0'].map({'l':1, 'j':2, 'k':3})
    int_cols = [x for x in retf.columns if x not in ['id','arch']]
    retf[int_cols] = retf[int_cols].astype(float)
    return retf

train_df = get_df(train_data)

base_cols = [x for x in train_df.columns if x[:3]=='col']
len(base_cols)

37

In [4]:
from scipy.special import erfinv 

for c in target_cols:
    train_df[c+'_log1p'] = train_df[c].map(lambda x: np.log1p(x))
    train_y=train_df[c]
    mmin=np.min(train_y)+1
    mmax=np.max(train_y)+1
    train_y=np.sqrt(2) * erfinv(2 * (train_y+mmin)/(mmin+mmax)-1)
    train_df[c+'_trans_y'] = train_y

In [5]:
def train_ohe(train_df, use_cols):
    for c in use_cols:
        for j in sorted(train_df[c].unique()):
            train_df[f'ohe_{c}_{int(j)}'] = np.where(train_df[c]==j, 1, 0)

In [6]:
train_ohe(train_df, base_cols)

In [8]:
len(base_cols)

37

In [9]:
ohe_cols = [x for x in train_df.columns if 'ohe' in x]

In [10]:
len(ohe_cols)

93

In [11]:
def get_step(train_df):
    res0 = []
    res1 = []
    res2 = []
    res3 = []
    res4 = []
    map_={'l':1, 'j':2, 'k':3}
    time_step=12
    head_ = set([1+_*3 for _ in range(12)])
    mlp_ = set([2+_*3 for _ in range(12)])
    emb_ = set([3+_*3 for _ in range(12)])
    depth_ = set([0])
    for item in train_df.arch:
        ret = np.array(list(item[1:]),dtype=np.float32).reshape(-1,3)
        res0.append([int(item[x]) if x in head_ else 0 for x in range(37)])
        res1.append([int(item[x]) if x in mlp_ else 0 for x in range(37)])
        res2.append([int(item[x]) if x in emb_ else 0 for x in range(37)])
        res3.append([map_[item[0]]]+[0]*36)
        res4.append([map_[item[0]]]+list(np.array(list(item[1:]),dtype=np.float32)))
        
    train_df['head'] = res0
    train_df['mlp'] = res1
    train_df['emb'] = res2
    train_df['depth'] = res3
    train_df['all_emb'] = res4

get_step(train_df)

In [13]:
len(train_df.all_emb.iloc[0])

37

In [12]:
import paddle
from paddle import nn

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\paddle\vision\transforms\functional_pil.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': Image.NEAREST,
C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\paddle\vision\transforms\functional_pil.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': Image.BILINEAR,
C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\paddle\vision\transforms\functional_pil.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (20

In [14]:
[x for x in train_df.columns if 'tran' in x]

['cplfw_rank_trans_y',
 'market1501_rank_trans_y',
 'dukemtmc_rank_trans_y',
 'msmt17_rank_trans_y',
 'veri_rank_trans_y',
 'vehicleid_rank_trans_y',
 'veriwild_rank_trans_y',
 'sop_rank_trans_y']

In [15]:
from sklearn.model_selection import KFold
from tqdm import tqdm_notebook as tqdm

#定义数据
class MyDataset(paddle.io.Dataset):
    def __init__(self, df, use_cols, target_cols, show=0, is_val=0):
        self.df = df
        self.show = show
        self.use_cols = use_cols
        self.target_cols = target_cols
        self.is_val = is_val

        self.prepare_data()
        
    def __len__(self):
        return self.df.shape[0]
    
    def prepare_data(self):
        self.inputs = self.df[self.use_cols].values
        if self.is_val==0:
            self.y = self.df[self.target_cols].values
            self.y0 = self.df[[self.target_cols[0]]].values
            self.y1 = self.df[[self.target_cols[1]]].values
            self.y2 = self.df[[self.target_cols[2]]].values
            self.y3 = self.df[[self.target_cols[3]]].values
            self.y4 = self.df[[self.target_cols[4]]].values
            self.y5 = self.df[[self.target_cols[5]]].values
            self.y6 = self.df[[self.target_cols[6]]].values
            self.y7 = self.df[[self.target_cols[7]]].values
        
        if self.show==1:
            print('inputs_shape',self.inputs.shape)
            if self.is_val==0:
                print('y_shape',self.y.shape)
        
    def __getitem__(self, idx):
        if self.is_val==0:
            data = {
                "input": paddle.to_tensor(self.inputs[idx], dtype='float32'),
                "y": paddle.to_tensor(self.y[idx], dtype='float32'),
                "y0": paddle.to_tensor(self.y0[idx], dtype='float32'),
                "y1": paddle.to_tensor(self.y1[idx], dtype='float32'),
                "y2": paddle.to_tensor(self.y2[idx], dtype='float32'),
                "y3": paddle.to_tensor(self.y3[idx], dtype='float32'),
                "y4": paddle.to_tensor(self.y4[idx], dtype='float32'),
                "y5": paddle.to_tensor(self.y5[idx], dtype='float32'),
                "y6": paddle.to_tensor(self.y6[idx], dtype='float32'),
                "y7": paddle.to_tensor(self.y7[idx], dtype='float32'),
            }
        else:
            data = {
                "input": paddle.to_tensor(self.inputs[idx], dtype='float32')
            }
        
        return data

In [16]:
train0 = train_df.sample(400, random_state=666).reset_index(drop=True)
val0 = train_df[~train_df.id.isin(train0.id)].reset_index(drop=True)

In [17]:
use_cols = ohe_cols

In [18]:
len(use_cols)

93

In [19]:
target_cols

['cplfw_rank',
 'market1501_rank',
 'dukemtmc_rank',
 'msmt17_rank',
 'veri_rank',
 'vehicleid_rank',
 'veriwild_rank',
 'sop_rank']

In [20]:
target_cols = [x+'_trans_y' for x in target_cols]
target_cols

['cplfw_rank_trans_y',
 'market1501_rank_trans_y',
 'dukemtmc_rank_trans_y',
 'msmt17_rank_trans_y',
 'veri_rank_trans_y',
 'vehicleid_rank_trans_y',
 'veriwild_rank_trans_y',
 'sop_rank_trans_y']

In [21]:
train_dataset = MyDataset(train0, use_cols, target_cols, 1)
val_dataset = MyDataset(val0, use_cols, target_cols, 1)

inputs_shape (400, 93)
y_shape (400, 8)
inputs_shape (100, 93)
y_shape (100, 8)


In [25]:
val_dataset[:3]['y']

Tensor(shape=[3, 8], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 0.94763881,  1.09971321,  0.25541422, -0.13297699, -0.54344082,
         -0.35504633, -0.83593118, -1.47683239],
        [ 0.60240257, -0.21938866,  0.06759532,  0.12288823,  0.25024846,
          1.07263899,  0.14813589, -1.06378710],
        [-0.72726136,  0.07762827,  0.48068762,  0.36038014, -0.38182011,
          1.66552341,  0.88668317,  0.26058680]])

In [26]:
val_dataset[:3]['y0']

Tensor(shape=[3, 1], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 0.94763881],
        [ 0.60240257],
        [-0.72726136]])

In [27]:
val_dataset[:3]['input']

Tensor(shape=[3, 93], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
         1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
         1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
         1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 1., 0.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
         0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
         0., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 

In [28]:
import gc
import os
import time
import random
import numpy as np
import paddle
from paddle import nn
from paddle.io import DataLoader

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    paddle.seed(seed)
    
def count_parameters(model, all=False):
    if all:
        return sum(p.numel() for p in model.parameters())
    else:
        return sum(p.numel() for p in model.parameters() if p.stop_gradient==False)


def save_model_weights(model, modelpath, filename):
    paddle.save(model.state_dict(), modelpath+filename)
    return f"\n -> Save weights to {modelpath+filename}\n"

from paddle.optimizer.lr import LRScheduler
#定义warmup
class LinearWarmup(LRScheduler):
    def __init__(self,
                 learning_rate,
                 warmup_steps,
                 num_training_steps,
                 end_lr=0.1,
                 start_lr=0.,
                 last_epoch=-1,
                 verbose=False):
        type_check = isinstance(learning_rate, float) or isinstance(
            learning_rate, int) or isinstance(learning_rate, LRScheduler)
        if not type_check:
            raise TypeError(
                "the type of learning_rate should be [int, float or LRScheduler], the current type is {}".
                format(learning_rate))
        self.learning_rate = learning_rate
        assert warmup_steps > 0 and isinstance(
            warmup_steps, int), " 'warmup_steps' must be a positive integer."
        self.warmup_steps = warmup_steps
        self.start_lr = start_lr
        self.end_lr = learning_rate
        assert end_lr > start_lr, "end_lr {} must be greater than start_lr {}".format(
            end_lr, start_lr)
        super(LinearWarmup, self).__init__(start_lr, last_epoch, verbose)
        self.num_training_steps = num_training_steps

    def state_dict(self):
        """
        Returns the state of the LinearWarmup scheduler as a :class:`dict`.
        It is a subset of ``self.__dict__`` .
        """
        state_dict = super(LinearWarmup, self).state_dict()
        if isinstance(self.learning_rate, LRScheduler):
            state_dict["LinearWarmup_LR"] = self.learning_rate.state_dict()
        return state_dict

    def set_state_dict(self, state_dict):
        """
        Loads state_dict for LinearWarmup scheduler.
        """
        super(LinearWarmup, self).set_state_dict(state_dict)
        if isinstance(self.learning_rate, LRScheduler):
            self.learning_rate.set_state_dict(state_dict["LinearWarmup_LR"])

    def get_lr(self):
        if self.last_epoch < self.warmup_steps:
            return (self.end_lr - self.start_lr) * float(
                self.last_epoch) / float(self.warmup_steps) + self.start_lr
        else:
            return (self.end_lr - self.start_lr) * max(
            0.0, float(self.num_training_steps - self.last_epoch) / float(max(1, self.num_training_steps - self.warmup_steps))
        )

In [29]:
import scipy

def compute_metric(pred, y):
    corr = []
    if pred.shape[1]>2:
        for i in range(8):
            corr.append(scipy.stats.stats.kendalltau(pred[:, i], y[:, i])[0])
    else:
        corr.append(scipy.stats.stats.kendalltau(pred, y)[0])
    return np.array(corr)

class CVPRLoss_tanh(nn.Layer):
    # kendall tanh
    def __init__(self):
        super(CVPRLoss_tanh, self).__init__()
        
    def forward(self, pred, y):
        return 1.-paddle.concat(
            [self.get_score(y[:,i], pred[:,i]) for i in range(y.shape[1])]
        )
        
    def get_score(self, actuals, preds):
        sa = actuals.argsort()
        tmp = preds.index_select(index=sa)
        score = paddle.concat([((tmp[i:]-tmp[i-1])).tanh() for i in range(1, tmp.shape[0])]).sum()
        score1 = score/sum(list(range(1,len(tmp))))
        return score1
    
class CVPRLoss_tanh1(nn.Layer):
    # kendall tanh
    def __init__(self):
        super(CVPRLoss_tanh1, self).__init__()
        
    def forward(self, pred, y):
        return 1.-paddle.concat(
            [self.get_score(pred[:,i], y[:,i]) for i in range(y.shape[1])]
        )

    def get_score(self, outputs, labels):
        output1 = paddle.tile(outputs.unsqueeze(1), repeat_times=(1, outputs.shape[0]))
        label1 = paddle.tile(labels.unsqueeze(1), repeat_times=(1, labels.shape[0]))

        tmp = ((output1-output1.t())*paddle.sign(label1-label1.t())).tanh()
        eye_tmp = tmp*paddle.eye(tmp.shape[0])
        new_tmp = tmp - eye_tmp
        
        loss = paddle.sum(new_tmp)/(outputs.shape[0]*(outputs.shape[0]-1))

        return loss

In [30]:
def train(model, 
        train_dataset, 
        val_dataset, 
        verbose=20, 
        fold_=0,
        modelname='MLP_base',
        modelpath=r'./model'+'//',
        input='input',
        y='y',
        early_stop_round=60,
        debug=False):
    
    print(f'Model parameters count: {count_parameters(model)}')
    #数据加载
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE_TEST,
        shuffle=False,
        drop_last=False,
    )
    print(f'train batch num: {len(train_loader)}')
    print(f'val batch num: {len(val_loader)}')
            

    # Scheduler
    num_warmup_steps = int(0.1 * EPOCHS * len(train_loader))
    num_training_steps = int(EPOCHS * len(train_loader))
    
    scheduler = LinearWarmup(
        learning_rate=LR,
        warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )
    # Optimizer
    optimizer = getattr(paddle.optimizer, optim)(learning_rate=scheduler, parameters=model.parameters())
    print(f'optim: {optim}, lr: {LR}, warmup_steps: {num_warmup_steps}, training steps: {num_training_steps}')
    
    print(f'early stopping round: {early_stop_round}\n')
    #train
    bst_epoch=0
    score_best=0
    first_epoch_eval=0
    for epoch in range(EPOCHS):
        if epoch > early_stop_round and (epoch - bst_epoch > early_stop_round):
            print(f'early stopping.')
            break
        
        model.train()
        optimizer.clear_grad()
        start_time = time.time()

        avg_loss = 0
        for data in train_loader:
            pred = model(data[input])
#             print(pred.shape,data['y'].shape)

            loss = loss_fct(
                pred,
                data[y]
            ).mean()

            loss.backward()
            avg_loss += loss.item() / len(train_loader)

            optimizer.step()
            scheduler.step()

            optimizer.clear_grad()

        #VAL
        model.eval()
        mae, avg_val_loss = 0, 0
        preds = []

        with paddle.no_grad():
            for data in val_loader:
                pred = model(data[input])

                loss = loss_fct(
                    pred,
                    data[y]
                ).mean()

                avg_val_loss += loss.item() / len(val_loader)

                preds.append(pred.numpy())

        preds = np.concatenate(preds, 0)
        if y=='y':
            mae = compute_metric(preds,val_dataset.df[target_cols].values).mean()
        else:
            mae = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
        
        elapsed_time = time.time() - start_time
        if (epoch + 1) % verbose == 0:
            elapsed_time = elapsed_time * verbose
            lr = scheduler.get_lr()
#             lr=LR
            print(
                f"Epoch {epoch + 1:02d}/{ EPOCHS:02d} \t lr={lr:.1e}\t t={elapsed_time:.0f}s \t"
                f"loss={avg_loss:.4f}",
                end="\t",
            )

            if (epoch + 1 >= first_epoch_eval) or (epoch + 1 == EPOCHS):
                print(f"val_loss={avg_val_loss:.4f}\tcorr={mae:.4f}")
            else:
                print("")
                
        #保存最优模型
        if mae>score_best:
            bst = save_model_weights(model, modelpath, f'{modelname}_{fold_}.pt')
            score_best = mae
            bst_epoch = epoch
            if y=='y':
                bst_list = compute_metric(preds,val_dataset.df[target_cols].values)
            else:
                bst_list = compute_metric(preds,val_dataset.df[[target_cols[int(y.replace('y',''))]]].values).mean()
            bst_preds = preds
    print(f'best score {score_best}, best epoch: {bst_epoch}, {bst} ' )
    print(np.mean(bst_list),bst_list,'\n\n')
    del (val_loader, train_loader, loss, data, pred)
    gc.collect()
    paddle.device.cuda.empty_cache()
    
    return bst_preds, bst_list

In [31]:
class CVPRModel(nn.Layer):
    def __init__(
        self,
        input_dim=3,
        num_classes=8,
        time_step=12,
        bi=True
    ):
        super(CVPRModel, self).__init__()
        
        self.Linear = nn.Linear(input_dim, num_classes)
        
    def forward(self, x):
        pred = self.Linear(x)
        return pred

In [32]:
#参数设置
seed = 666
BATCH_SIZE = 400
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.001
optim = "Adam"
paddle.set_device("gpu")
modelpath = r'./model'+'//'

In [33]:
a = paddle.rand(shape=(128,93))
net = CVPRModel(input_dim=93,
                num_classes=len(target_cols),
                bi=True,
                time_step=37
               )
b = net(a)
b.shape

[128, 8]

In [34]:
b.flatten(1).shape

[128, 8]

In [35]:
del a,b,net;
gc.collect()
paddle.device.cuda.empty_cache()

In [38]:
seed = 666
BATCH_SIZE = 400
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.1
optim = "Adam"
paddle.set_device("gpu")
modelpath = r'E:\cvpr_model'+'//'

loss_fct = CVPRLoss_tanh1()
loss_fct = nn.MSELoss()
index = '_t'
seed_everything(seed)
modelname = 'paddle_test'
model = CVPRModel(input_dim=len(use_cols),
                num_classes=8,
                bi=True,
                time_step=37
               )
preds,_ = train(model, 
            train_dataset, 
            val_dataset, 
            verbose=20, 
            fold_=index,
            modelname=modelname,
            modelpath=modelpath,
            input='input',
            y='y',
            early_stop_round=60,
            debug=False
             )

Model parameters count: Tensor(shape=[1], dtype=int64, place=CUDAPlace(0), stop_gradient=False,
       [752])
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.1, warmup_steps: 30, training steps: 300
early stopping round: 60

Epoch 20/300 	 lr=6.7e-02	 t=8s 	loss=0.3478	val_loss=0.3214	corr=0.6813
Epoch 40/300 	 lr=9.6e-02	 t=6s 	loss=0.1772	val_loss=0.1973	corr=0.7555
Epoch 60/300 	 lr=8.9e-02	 t=8s 	loss=0.1392	val_loss=0.1664	corr=0.7801
Epoch 80/300 	 lr=8.1e-02	 t=7s 	loss=0.1338	val_loss=0.1589	corr=0.7947
Epoch 100/300 	 lr=7.4e-02	 t=6s 	loss=0.1332	val_loss=0.1574	corr=0.7939
Epoch 120/300 	 lr=6.7e-02	 t=5s 	loss=0.1329	val_loss=0.1578	corr=0.7938
early stopping.
best score 0.7948484848484849, best epoch: 78, 
 -> Save weights to E:\cvpr_model//paddle_test__t.pt
 
0.7948484848484849 [0.35919192 0.87151515 0.88808081 0.9640404  0.88484848 0.68161616
 0.90787879 0.80161616] 




In [ ]:
# best score 0.7948484848484849, best epoch: 78, 
#  -> Save weights to E:\cvpr_model//paddle_test__t.pt
 
# 0.7948484848484849 [0.35919192 0.87151515 0.88808081 0.9640404  0.88484848 0.68161616
#  0.90787879 0.80161616] 

In [39]:
seed = 666
BATCH_SIZE = 400
BATCH_SIZE_TEST = 128
EPOCHS = 300
LR = 0.1
optim = "Adam"
paddle.set_device("gpu")
modelpath = r'E:\cvpr_model'+'//'

# K折训练，同时
loss_fct = CVPRLoss_tanh()
loss_fct = nn.MSELoss()
k=5
scoref = []
skf = KFold(n_splits=k, shuffle=False)
for index, (train_index, test_index) in enumerate(skf.split(train_df)):   
    print(f'FOLD {index}')
    train0 = train_df.iloc[train_index]
    val0 = train_df.iloc[test_index]   
    train_dataset = MyDataset(train0, use_cols, target_cols)
    val_dataset = MyDataset(val0, use_cols, target_cols)
    print(f'train size: {len(train0)}, val size: {len(val0)}')

    modelname = f'paddle_super_linear_mse'
    seed_everything(seed)
    model = CVPRModel(input_dim=len(use_cols),
                    num_classes=8,
                    bi=True,
                    time_step=37
                   )
    preds,_ = train(model, 
                train_dataset, 
                val_dataset, 
                verbose=20, 
                fold_=index,
                modelname=modelname,
                modelpath=modelpath,
                input='input',
                y='y',
                early_stop_round=60,
                debug=False
                 )
    scoref.append(_)
scoreff = scoref
np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

FOLD 0
train size: 400, val size: 100
Model parameters count: Tensor(shape=[1], dtype=int64, place=CUDAPlace(0), stop_gradient=False,
       [752])
train batch num: 1
val batch num: 1
optim: Adam, lr: 0.1, warmup_steps: 30, training steps: 300
early stopping round: 60

Epoch 20/300 	 lr=6.7e-02	 t=8s 	loss=0.3333	val_loss=0.4005	corr=0.6499
Epoch 40/300 	 lr=9.6e-02	 t=7s 	loss=0.1816	val_loss=0.2423	corr=0.7374
Epoch 60/300 	 lr=8.9e-02	 t=7s 	loss=0.1364	val_loss=0.1754	corr=0.7774
Epoch 80/300 	 lr=8.1e-02	 t=6s 	loss=0.1311	val_loss=0.1725	corr=0.7855
Epoch 100/300 	 lr=7.4e-02	 t=6s 	loss=0.1305	val_loss=0.1709	corr=0.7886
Epoch 120/300 	 lr=6.7e-02	 t=9s 	loss=0.1304	val_loss=0.1707	corr=0.7878
Epoch 140/300 	 lr=5.9e-02	 t=7s 	loss=0.1304	val_loss=0.1706	corr=0.7885
Epoch 160/300 	 lr=5.2e-02	 t=7s 	loss=0.1304	val_loss=0.1706	corr=0.7881
early stopping.
best score 0.788888888888889, best epoch: 106, 
 -> Save weights to E:\cvpr_model//paddle_super_linear_mse_0.pt
 
0.7888888888

(0.78087,
 [0.26432, 0.8678, 0.89648, 0.96024, 0.89228, 0.66602, 0.91327, 0.78651])

In [40]:
np.round(np.array(scoreff).mean(1).mean(), 5), [round(x, 5) for x in np.array(scoreff).mean(0)]

(0.78087,
 [0.26432, 0.8678, 0.89648, 0.96024, 0.89228, 0.66602, 0.91327, 0.78651])

加载test
______________________________________________________________________________________________________

In [41]:
with open('./data/CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)
test_data['arch99997']

{'cplfw_rank': 0,
 'market1501_rank': 0,
 'dukemtmc_rank': 0,
 'msmt17_rank': 0,
 'veri_rank': 0,
 'vehicleid_rank': 0,
 'veriwild_rank': 0,
 'sop_rank': 0,
 'arch': 'j121221121221221311331321121221000000'}

In [44]:
target_cols = [x.replace('_trans_y','') for x in target_cols]

In [45]:
test_df = get_df(test_data)
get_step(test_df)

In [46]:
len(base_cols)

37

In [47]:
train_ohe(test_df, base_cols)

In [48]:
len([x for x in test_df if x[:3]=='ohe'])

93

In [49]:
def pred_cuda(test_df, model):
    #获得预测
    test_dataset = MyDataset(test_df, use_cols, target_cols, 0)
    test_loader = DataLoader(
        test_dataset,
        batch_size=1024*128,
        shuffle=False,
        drop_last=False
    )

    preds = []
    model.eval()
    with paddle.no_grad():
        for data in tqdm(test_loader):
            pred = model(data['input'])
            preds.append(pred.numpy())
    preds = np.concatenate(preds, 0)
    print(preds.shape)
    
    del test_dataset, test_loader, model;
    gc.collect()
    paddle.device.cuda.empty_cache()
    return preds

def predict_all(test_df, k=5, modelname = f'paddle_7937_LSTM_2layer_hardtanh'):
    print(f'Model {modelname}')
    #pred
    cols = []
    for fold_ in range(k):
        model = CVPRModel(input_dim=93,
                    num_classes=8,
                    bi=True,
                    time_step=37
                   )
        state_dict = paddle.load(modelpath+f'{modelname}_{fold_}.pt')
        model.set_state_dict(state_dict)
        
        pred_ = pred_cuda(test_df, model)
        tmp_c = [f'{target}_{fold_}' for target in target_cols]
        test_df[tmp_c] = pred_
        cols += tmp_c
    #get rank
    print(cols)
    test_df[cols] = test_df[cols].rank()
    for c in target_cols:
        test_df[c] = test_df[[f'{c}_{fold_}' for fold_ in range(k)]].mean(axis=1)
        test_df[c] = test_df[c].rank()

In [50]:
k, modelname

(5, 'paddle_super_linear_mse')

In [51]:
model_dt = predict_all(test_df, k=k, modelname='paddle_super_linear_mse')

Model paddle_super_linear_mse


C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipywidgets\widgets\widget_selection.py:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Iterable
C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/1 [00:00<?, ?it/s]

...

(99500, 8)


  0%|          | 0/1 [00:00<?, ?it/s]

(99500, 8)


  0%|          | 0/1 [00:00<?, ?it/s]

(99500, 8)
['cplfw_rank_0', 'market1501_rank_0', 'dukemtmc_rank_0', 'msmt17_rank_0', 'veri_rank_0', 'vehicleid_rank_0', 'veriwild_rank_0', 'sop_rank_0', 'cplfw_rank_1', 'market1501_rank_1', 'dukemtmc_rank_1', 'msmt17_rank_1', 'veri_rank_1', 'vehicleid_rank_1', 'veriwild_rank_1', 'sop_rank_1', 'cplfw_rank_2', 'market1501_rank_2', 'dukemtmc_rank_2', 'msmt17_rank_2', 'veri_rank_2', 'vehicleid_rank_2', 'veriwild_rank_2', 'sop_rank_2', 'cplfw_rank_3', 'market1501_rank_3', 'dukemtmc_rank_3', 'msmt17_rank_3', 'veri_rank_3', 'vehicleid_rank_3', 'veriwild_rank_3', 'sop_rank_3', 'cplfw_rank_4', 'market1501_rank_4', 'dukemtmc_rank_4', 'msmt17_rank_4', 'veri_rank_4', 'vehicleid_rank_4', 'veriwild_rank_4', 'sop_rank_4']


In [52]:
test_df[target_cols] = test_df[target_cols].astype(int)-1

In [53]:
test_df[target_cols].describe()

,cplfw_rank,market1501_rank,dukemtmc_rank,msmt17_rank,veri_rank,vehicleid_rank,veriwild_rank,sop_rank
count,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000,99500.000000
mean,49749.418402,49749.418040,49749.418633,49749.419809,49749.419578,49749.419186,49749.419146,49749.419719
std,28723.319508,28723.320886,28723.319596,28723.320887,28723.321110,28723.320144,28723.319614,28723.320946
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24874.750000,24874.750000,24874.500000,24874.750000,24874.750000,24874.750000,24874.750000,24874.750000
50%,49749.000000,49749.500000,49749.500000,49749.500000,49749.500000,49749.000000,49749.500000,49749.500000
75%,74624.250000,74623.500000,74624.250000,74624.250000,74624.250000,74624.250000,74624.250000,74624.000000
max,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000,99499.000000


In [54]:
modelname

'paddle_super_linear_mse'

In [55]:
modelname = 'paddle_super_linear_mse'

In [56]:
#输出
for i in tqdm(test_df[['id']+target_cols].values):
    id_ = i[0]
    for k,v in enumerate(target_cols):
        k += 1
        test_data[id_][v] = i[k]
        
with open(f'./sub/{modelname}.json', 'w') as f:
    json.dump(test_data, f)

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/99500 [00:00<?, ?it/s]

In [57]:
modelname

'paddle_super_linear_mse'